In [ ]:
import sys
import glob
import os
sys.path.append(r'/dls/science/groups/e02/Chris/Code/GitHub/Merlin-Medipix')  # location of epsic_tools 
#'Z:/Chris/Code/GitHub/Merlin-Medipix')
#r'/dls/science/groups/e02/Chris/Code/GitHub/Merlin-Medipix') # location of epsic_tools 
import hyperspy.api as hs
import epsic_tools.api as epsic
from epsic_tools.toolbox.ptychography import ptyrex
import matplotlib.pyplot as plt
import numpy as np

ep = epsic.ptychography2D.Ptychography2D()

In [ ]:
%matplotlib qt

In [ ]:
#alternitavely pull path and ID from crop_data and Save_and_run
save_path = '/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C'
experiment_ID = 'scancrop_64_Au'
path = save_path + '/' + experiment_ID + '/processing/pycho'
crop_to= 500
ep.load_series(path, crop_to = crop_to, verbose = True)

In [ ]:
#list files in path
#path = '/dls/e02/data/2019/cm22979-6/processing/Merlin/Merlin/20191001_15kV_ptycho/MoS2_700C//scancrop_64_Au/processing/pycho'
print(path)
f = glob.glob(path + '/*.hdf')
for index, value in enumerate(f): 
    print('[',index,']', os.path.split(value)[1]) 

In [ ]:
#load a single reconstruction
n = 1
ep = epsic.ptychography2D.Ptychography2D()
print(f[n])
ep.load_recon(f[n])
#print(ep.get_detector_orientation())

In [ ]:
#plot some things
ep.recon_object.plot()
ep.recon_probe.plot()
ep.recon_error.plot()
#print(ep.get_detector_orientation())

In [ ]:
#plot object, fft, probe and error on one plot 
#get dat
#plot object phase
crop_size = 500
contrast = 5
fig_width =3.6
fig1 = plt.figure(figsize = (fig_width,  4.5*fig_width))

ax1 = plt.subplot(421)
ax1.set_axis_off()
ax2 = plt.subplot(422)
ax2.set_axis_off()
ax3 = plt.subplot(423)
ax3.set_axis_off()
ax4 = plt.subplot(424)
ax4.set_axis_off()

ax5 = plt.subplot(425)
ax5.set_axis_off()
ax6 = plt.subplot(426)
ax6.set_axis_off()
ax7 = plt.subplot(414)

obj_size = ep.recon_object.inav[0].data.shape
from_ind = obj_size[0] / 2 - crop_size / 2 
to_ind =  obj_size[0] / 2 + crop_size / 2
obj_crop = ep.recon_object.isig[from_ind:to_ind, from_ind:to_ind ]
max_grey = obj_crop.inav[0].data.max() / contrast
min_grey = obj_crop.inav[0].data.min() / contrast
ax1.imshow(obj_crop.inav[0].data, vmin = min_grey ,vmax =  max_grey )#, sharex = ax3)

#calc fftigsize=(15,15)
phase_fft = np.fft.fft2(obj_crop.inav[0].data)
phase_fft =np.fft.fftshift(phase_fft)
phase_fft =np.log10(np.abs(phase_fft)**2)
#plot fftplt.tight_layout()
#ax2 = plt.subplot(3,2,2)
ax2.imshow(phase_fft)#, sharex = ax4, sharey = ax1)


#plot object mod
ax3.imshow(obj_crop.inav[1].data)
#calc fft
phase_fft =np.fft.fft2(obj_crop.inav[1].data)
phase_fft =np.fft.fftshift(phase_fft)
phase_fft =np.log10(np.abs(phase_fft)**2)
#plot fft
ax4.imshow(phase_fft)
ax5.imshow(ep.recon_probe.inav[0].data)
#plot probe modsharex
ax6.imshow(ep.recon_probe.inav[1].data)
#igsize=(15,15)
#plot error function
#plt.subplot(ax[2][:])
ax7.plot(ep.recon_error.data)
ax7.ticklabel_format(axis='y', style='sci', scilimits = (3,0))#, scilimits=None, useOffset=None, useLocale=None, useMathText=None)

#
fig_title = os.path.split(f[n])[1]
fig1.suptitle(fig_title, fontsize = 'small')
plt.tight_layout(rect=[0, 0.03, 1, 0.95], pad = 2)

In [ ]:
#save plot
fig_save_path = path + '/figs//' 
if not os.path.isdir(fig_save_path):
    os.mkdir(fig_save_path)
fig_save_name = fig_save_path + fig_title +'.png'
plt.savefig(fig_save_name, dpi = 4* crop_size / fig_width )
print('fig saved to : ', fig_save_name)